# Attention mechanism 

- Seq2Seq 모델의 문제점
    - Seq2Seq 모델은 Encoder에서 입력 시퀀스에 대한 특성을 **하나의 고정된 context vector**에 압축하여 Decoder로 전달 한다. Decoder는 이 context vector를 이용해서 출력 시퀀스를 만든다.
    - 하나의 고정된 크기의 vector에 모든 입력 시퀀스의 정보를 넣다보니 정보 손실이 발생한다.
    - Decoder에서 출력 시퀀스를 생성할 때 동일한 context vector를 기반으로 한다. 그러나 각 생성 토큰마다 입력 시퀀스에서 참조해야 할 중요도가 다를 수 있다. seq2seq는 encoder의 마지막 hidden state를 context로 받은 뒤 그것을 이용해 모든 출력 단어들을 생성하므로 그 중요도에 대한 반영이 안된다.

## Attention Mechanism 아이디어
-  Decoder에서 출력 단어를 예측하는 매 시점(time step)마다, Encoder의 입력 문장(context vector)을 다시 참고 하자는 것. 이때 전체 입력 문장의 단어들을 동일한 비율로 참고하는 것이 아니라, Decoder가 해당 시점(time step)에서 예측해야할 단어와 연관이 있는 입력 부분을 좀 더 집중(attention)해서 참고 할 수 있도록 하자는 것이 기본 아이디어이다.
- 다양한 Attention 종류들이 있다.
    -  Decoder에서 출력 단어를 예측하는 매 시점(time step)마다 Encoder의 입력 문장의 어느 부분에 더 집중(attention) 할지를 계산하는 방식에 따라 다양한 attention 기법이 있다.
    -  `dot attention - Luong`, `scaled dot attention - Vaswani`, `general  attention - Luong`, `concat  attention - Bahdanau` 등이 있다.

# Data Loading

In [ ]:
import os
import numpy as np
import pandas as pd
import random

df = pd.read_csv('datasets/ChatbotData.csv')
df

# 토큰화

## 학습

## 저장

# Dataset 생성
- 한문장 단위로 학습시킬 것이므로 DataLoader를 생성하지 않고 Dataset에서 index로 조회한 질문-답변을 학습시킨다.

In [ ]:
import random
import os
import time
import math

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import optim

device = "cuda" if torch.cuda.is_available() else "cpu"
device

### Dataset 클래스 정의

In [ ]:
class ChatbotDataset(Dataset):

    """
    Attribute
        max_length
        tokenizer: Tokenizer
        vocab_size: int - Tokenizer에 등록된 총 어휘수
        SOS: int - [SOS] 문장의 시작 토큰 id
        EOS: int = [EOS] 문장의 끝 토큰 id
        question_squences: list - 모든 질문 str을 token_id_list(token sequence) 로 변환하여 저장한 list 
        answser_sequences: list - 모든 답변 str을 token_id_list(token sequence) 로 변환하여 저장한 list.
    """
    def __init__(self, question_texts, answer_texts, tokenizer, min_length=3, max_length=25):
        """
        question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
        answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1", "답2", ...]
        tokenizer: Tokenizer
        min_length=3: int - 최소 토큰 개수. 질문과 답변의 token수가 min_length 이상인 것만 학습한다.
        max_length=25:int 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        """
        
   

    def __add_special_tokens(self, token_sequence):
        """
        max_length 보다 token 수가 많은 경우 max_length에 맞춰 뒤를 잘라낸다.
        token_id_list 에 [EOS] 토큰 추가. 
        """
        pass

    
    def __process_sequence(self, text):
        """
        한 문장 string을 받아서 token_id 리스트(list)로 변환 후 반환
        """
       pass

    
    def __len__(self):
        pass


    def __getitem__(self, index):        
        pass

### Dataset 객체 생성

# 모델

## Encoder
- seq2seq 모델과 동일 한 구조
    - 이전 코드(seq2seq)와 비교해서 forward()에서 입력 처리는 token 하나씩 하나씩 처리한다. 

![encoder](figures/attn_encoder-network_graph.png)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        """
        Parameter
            num_vocabs: int - 총 어휘수 
            hidden_size: int - GRU의 hidden size
            embedding_dim: int - Embedding vector의 차원수 
            num_layers: int - GRU의 layer수
        """
        pass

    
    def forward(self, x, hidden):
        """
        질문의 token한개의 토큰 id를 입력받아 hidden state를 출력
        Parameter
            x: 한개 토큰. shape-[1]
            hidden: hidden state (이전 처리결과). shape: [1, 1, hidden_size]
        Return
            tuple: (output, hidden) - output: [1, 1, hidden_size],  hidden: [1, 1, hidden_size]
        """
        pass

    
    def init_hidden(self, device):
        """
        처음 입력할 hidden_state. 
        값: 0
        shape: (Bidirectional(1) x number of layers(1), batch_size: 1, hidden_size) 
        """
        pass

## Attention 적용 Decoder
![seq2seq attention outline](figures/attn_seq2seq_attention_outline.png)

- Attention은 Decoder 네트워크가 순차적으로 다음 단어를 생성하는 자기 출력의 모든 단계에서 인코더 출력 중 연관있는 부분에 **집중(attention)** 할 수 있게 한다. 
- 다양한 어텐션 기법중에 **Luong attention** 방법은 다음과 같다.
  
![attention decoder](figures/attn_decoder-network_graph.png)

### Attetion Weight
- Decoder가 현재 timestep의 단어(token)을 생성할 때 Encoder의 output 들 중 어떤 단어에 좀더 집중해야 하는지 계산하기 위한 가중치값.
  
![Attention Weight](figures/attn_attention_weight.png)

### Attention Value
- Decoder에서 현재 timestep의 단어를 추출할 때 사용할 Context Vector. 
    - Encoder의 output 들에 Attention Weight를 곱한다.
    - Attention Value는 Decoder에서 단어를 생성할 때 encoder output의 어떤 단어에 더 집중하고 덜 집중할지를 가지는 값이다.

![attention value](figures/attn_attention_value.png)

### Feature Extraction
- Decoder의 embedding vector와 Attention Value 를 합쳐 RNN(GRU)의 입력을 만든다.
    - **단어를 생성하기 위해 이전 timestep에서 추론한 단어(현재 timestep의 input)** 와 **Encoder output에 attention이 적용된 값** 이 둘을 합쳐 입력한다.
    - 이 값을 Linear Layer함수+ReLU를 이용해 RNN input_size에 맞춰 준다. (어떻게 input_size에 맞출지도 학습시키기 위해 Linear Layer이용)

![rnn](figures/att_attention_combine.png)

### 단어 예측(생성)
- RNN에서 찾은 Feature를 총 단어개수의 units을 출력하는 Linear에 입력해 **다음 단어를 추론한다.**
- 추론한 단어는 다음 timestep의 입력($X_t$)으로 RNN의 hidden은 다음 timestep 의 hidden state ($h_{t-1}$) 로 입력된다.


In [ ]:
class AttentionDecoder(nn.Module):

    def __init__(self, num_vocabs, hidden_size, embedding_dim, dropout_p=0.1, max_length=MAX_LENGTH):
        super().__init__()
        

    def forward(self, x, hidden, encoder_outputs):
        """
        Parameter
            x: 현재 timestep의 입력 토큰 id
            hidden: 이전 timestep 처리결과 hidden state
            encoder_outputs: Encoder output들. 
        Return
            tupe: (output, hidden, attention_weight)
                output: 예측한 단어별 다음 단어일 확률.  shape: [vocab_size]
                hidden: hidden_state. shape: [1, 1, hidden_size]
                atttention_weight: Encoder output 중 어느 단어에 집중해야하는 지 가중치값. shape: [1, max_length]
        
        현재 timestep 입력과 이전 timestep 처리결과를 기준으로 encoder_output와 계산해서  encoder_output에서 집중(attention)해야할 attention value를 계산한다.
        attention value와 현재 timestep 입력을 기준으로 단어를 추론(생성) 한다.
        """
        
        pass

    def initHidden(self, device):
        pass

# Training

In [ ]:
SOS_TOKEN = dataset.tokenizer.token_to_id("[SOS]")
EOS_TOKEN = dataset.tokenizer.token_to_id("[EOS]")

In [ ]:
# 한개 question-answer set 학습
def train(input_tensor, target_tensor, encoder, decoder, 
          encoder_optimizer, decoder_optimizer, 
          loss_fn, device, max_length=MAX_LENGTH, teacher_forcing_ratio=0.9):
    
    
    pass

In [ ]:
def train_iterations(encoder, decoder, n_iters, dataset, device, log_interval=1000, learning_rate=0.001):
    
    pass

  

## Model 생성, 학습

## 저장

## 검증